<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第三章：编码注意力机制

本notebook会用到以下第三方库

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.2.1+cu118


- 本章节覆盖LLMs的引擎--注意力机制

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">
</p>

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">
</p>

## 3.1 建模长序列的问题
- 本小结没有代码
- 在不同语种文本翻译任务中，因为不同语言的语法结构不同导致逐词翻译是不可行的

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">
</p>

- 在transformer模型出现之前，编码器-解码器RNN通常用于机器翻译任务
- 在这种设置中，编码器处理源语言的标记序列，使用隐藏状态（神经网络内部的一种中间层）来生成整个输入序列的浓缩表示

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="600px">
</p>

## 3.2 使用注意力机制捕捉数据之间的依赖
- 本小结没有代码
- 通过注意力机制，网络的文本生成解码器部分能够选择性地访问所有输入标记，这意味着在生成特定输出标记时，某些输入标记比其他标记更重要

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="600px">
</p>

- 自注意力在transformer中是一种旨在增强输入表示的技术，它使序列中的每个位置都能与同一序列中的所有其他位置进行交互并确定它们的相关性

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">
</p>

## 3.3 通过自注意力关注输入的不同部分

### 3.3.1 一个不带可训练权重的简单自注意力机制
- 本节解释了一个非常简化的自注意力变体，它不包含任何可训练的权重
- 这纯粹是为了说明目的，而不是transformer中使用的注意力机制
- 下一节，3.3.2节，将扩展这个简单的注意力机制来实现真正的自注意力机制
- 假设我们有一个输入序列$x^{(1)}$到$x^{(T)}$
  - 输入是一段文本（例如，一个句子如"Your journey starts with one step"），已经按照第2章所述转换为标记嵌入
  - 例如，$x^{(1)}$是表示单词"Your"的d维向量，以此类推
- **目标：**为输入序列$x^{(1)}$到$x^{(T)}$中的每个输入序列元素$x^{(i)}$计算上下文向量$z^{(i)}$（其中$z$和$x$具有相同的维度）
    - 上下文向量$z^{(i)}$是输入$x^{(1)}$到$x^{(T)}$的加权和
    - 上下文向量对特定输入是"上下文"特定的
      - 不用$x^{(i)}$作为任意输入标记的占位符，关于第二个输入$x^{(2)}$
      - 继续使用具体示例，不用占位符$z^{(i)}$，考虑第二个输出上下文向量$z^{(2)}$
      - 第二个上下文向量$z^{(2)}$是所有输入$x^{(1)}$到$x^{(T)}$的加权和，权重是相对于第二个输入元素$x^{(2)}$的
      - 注意力权重是决定在计算$z^{(2)}$时每个输入元素贡献多少的权重
      - 简而言之，可以将$z^{(2)}$视为$x^{(2)}$的修改版本，它还包含了与给定任务相关的所有其他输入元素的信息

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">
</p>

- （请注意，此图中的数字截断为小数点后一位，以减少视觉混乱；类似地，其他图表也可能包含截断的值）

- 按照惯例，未归一化的注意力权重被称为"注意力分数"，而归一化后和为1的注意力分数则被称为"注意力权重"

- 下面的代码逐步解释了上图中的过程

<br>

- 第一步：计算未归一化的注意力分数 $\omega$
- 假设使用第二个input token作为query，即 $q^{(2)} = x^{(2)}$，通过以下点乘操作计算为归一化分数
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 上面的 $\omega$ 是希腊字母"omega"，用来表示未归一化的注意力分数
    - 下标"21"在$\omega_{21}$中表示输入序列元素2被用作针对输入序列元素1的查询

- 假设有以下输入句子，已经按照第3章所述嵌入到3维向量中（这里为了说明目的使用非常小的嵌入维度，这样它可以在页面上不换行地显示）

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- （在本书中遵循常见的机器学习和深度学习惯例，其中训练样本表示为行，特征值表示为列；在上面显示的张量中，每一行代表一个词，每一列代表一个嵌入维度）

- 本节的主要目标是演示如何使用第二个输入序列$x^{(2)}$作为查询来计算上下文向量$z^{(2)}$

- 图中描述了这个过程的初始步骤，即通过点积操作计算$x^{(2)}$与所有其他输入元素之间的注意力分数ω

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">
</p>

- 使用输入序列元素2，$x^{(2)}$，作为示例来计算上下文向量$z^{(2)}$；在本节后面，将推广这一点来计算所有上下文向量。

- 第一步是通过计算查询$x^{(2)}$与所有其他输入标记之间的点积来计算未归一化的注意力分数

In [3]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 附注：点积本质上是将两个向量按元素相乘并对结果求和的简写

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)
